# Welcome to the ProgPy Playback Example!

# Introduction

In this Jupyter Notebook, we demonstrate the application of state estimation and prediction using playback data with the help of the `progpy` library. Our model utilizes the [BatteryCircuit](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#battery-model) model from the library to set the stage for our analysis.

Method: An instance of the `BatteryCircuit` model in progpy is created, the state estimation is set up by defining a state_estimator, and the prediction method is set up by defining a predictor.
        Prediction is then performed using playback data. For each data point:
1) The necessary data is extracted (time, current load, output values) and corresponding values defined (t, i, and z)
2) The current state estimate is performed and samples are drawn from this distribution
3) Prediction performed to get future states (with uncertainty) and the times at which the event threshold will be reached


By the end of this process, we anticipate having the following results:
1. Predicted future values (inputs, states, outputs, event_states) with an associated uncertainty.
2. The time of event occurrence with an associated uncertainty.
3. Various prediction metrics.
4. Illustrative figures representing our results.


### Importing Modules

In [ ]:
import csv
import numpy as np
from progpy.predictors import ToEPredictionProfile
from progpy.uncertain_data.multivariate_normal_dist import MultivariateNormalDist
from progpy.models import BatteryCircuit as Battery
from progpy.state_estimators import UnscentedKalmanFilter as StateEstimator
from progpy.predictors import UnscentedTransformPredictor as Predictor

Furthermore, let's set up some constants that we'll use later on:

In [ ]:
# Constants
NUM_SAMPLES = 20
NUM_PARTICLES = 1000 # For state estimator (if using ParticleFilter)
TIME_STEP = 1
PREDICTION_UPDATE_FREQ = 50 # Number of steps between prediction update
PLOT = True
PROCESS_NOISE = 1e-4 # Percentage process noise
MEASUREMENT_NOISE = 1e-4 # Percentage measurement noise
X0_COV = 1 # Covariance percentage with initial state
GROUND_TRUTH = {'EOD':2780}
ALPHA = 0.05
BETA = 0.90
LAMBDA_VALUE = 1500

First, we set up our model using the `BatteryCircuit` class.

In [ ]:
batt = Battery()

We then initialize our model's state and specify the process and measurement noise parameters based on the initial state values.

In [ ]:
x0 = batt.initialize()
batt.parameters['process_noise'] = {key: PROCESS_NOISE * value for key, value in x0.items()}
z0 = batt.output(x0)
batt.parameters['measurement_noise'] = {key: MEASUREMENT_NOISE * value for key, value in z0.items()}

The initial state is assumed to follow a multivariate normal distribution, with the covariance being a certain percentage of the absolute values of the initial states.

In [ ]:
x0 = MultivariateNormalDist(x0.keys(), list(x0.values()), np.diag([max(1e-9, X0_COV * abs(x)) for x in x0.values()]))

Next, we set up our state estimation using the `UnscentedKalmanFilter` method (or `ParticleFilter` method depending on the commented code). This method takes in our model, initial state, and the number of particles.

In [ ]:
filt = StateEstimator(batt, x0, num_particles = NUM_PARTICLES)

Finally, before we run our playback, we set up our prediction using the `UnscentedTransformPredictor` method (or `MonteCarlo` method depending on the commented code). We define the future load function to return a constant load, and we compute matrices `Q` and `R` based on the process noise and measurement noise parameters, respectively.

In [ ]:
load = batt.InputContainer({'i': 2.35})
def future_loading(t, x=None):
    return load
Q = np.diag([batt.parameters['process_noise'][key] for key in batt.states])
R = np.diag([batt.parameters['measurement_noise'][key] for key in batt.outputs])
mc = Predictor(batt, Q = Q, R = R)

### Running Playback

We start by initializing a step counter and creating an instance of `ToEPredictionProfile`.

In [ ]:
step = 0
profile = ToEPredictionProfile()

Next, we open our data file (`data_const_load.csv`) and read it line by line. For each line, we extract the time, current, temperature, and voltage. These values are then used to perform state estimation and, at certain intervals, prediction.

In [ ]:
with open('data_const_load.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip header
    for row in reader:
        step += 1
        print("{} s: {} W, {} C, {} V".format(*row))
        t = float(row[0])
        i = {'i': float(row[1])/float(row[3])}
        z = {'t': float(row[2]), 'v': float(row[3])}

        # State Estimation Step
        filt.estimate(t, i, z) 
        eod = batt.event_state(filt.x.mean)['EOD']
        print("  - Event State: ", eod)

        # Prediction Step (every PREDICTION_UPDATE_FREQ steps)
        if (step%PREDICTION_UPDATE_FREQ == 0):
            mc_results = mc.predict(filt.x, future_loading, t0 = t, n_samples=NUM_SAMPLES, dt=TIME_STEP)
            metrics = mc_results.time_of_event.metrics()
            print('  - ToE: {} (sigma: {})'.format(metrics['EOD']['mean'], metrics['EOD']['std']))
            profile.add_prediction(t, mc_results.time_of_event)


After completing the loop, we calculate various metrics such as the Prognostic Horizon, Alpha Lambda, and Cumulative Relative Accuracy. We also generate plots to illustrate the playback results.

In [ ]:
# Calculating Prognostic Horizon once the loop completes
from progpy.uncertain_data.uncertain_data import UncertainData
from progpy.metrics import samples as metrics

def criteria_eqn(tte : UncertainData, ground_truth_tte : dict) -> dict:
    """
    Sample criteria equation for playback. 
    # UPDATE THIS CRITERIA EQN AND WHAT IS CALCULATED

    Args:
        tte : UncertainData
            Time to event in UncertainData format.
        ground_truth_tte : dict
            Dictionary of ground truth of time to event.
    """

    # Set an alpha value
    bounds = {}
    for key, value in ground_truth_tte.items():
        # Set bounds for percentage_in_bounds by adding/subtracting to the ground_truth
        alpha_calc = value * ALPHA
        bounds[key] = [value - alpha_calc, value + alpha_calc] # Construct bounds for all events
    percentage_in_bounds = tte.percentage_in_bounds(bounds)

    # Verify if percentage in bounds for this ground truth meets beta distribution percentage limit
    return {key: percentage_in_bounds[key] > BETA for key in percentage_in_bounds.keys()}

# Generate plots for playback example
playback_plots = profile.plot(GROUND_TRUTH, ALPHA, True)

# Calculate prognostic horizon with ground truth, and print
ph = profile.prognostic_horizon(criteria_eqn, GROUND_TRUTH)
print(f"Prognostic Horizon for 'EOD': {ph['EOD']}")

# Calculate alpha lambda with ground truth, lambda, alpha, and beta, and print
al = profile.alpha_lambda(GROUND_TRUTH, LAMBDA_VALUE, ALPHA, BETA)
print(f"Alpha Lambda for 'EOD': {al['EOD']}")

# Calculate cumulative relative accuracy with ground truth, and print
cra = profile.cumulative_relative_accuracy(GROUND_TRUTH)
print(f"Cumulative Relative Accuracy for 'EOD': {cra['EOD']}")


## Conclusion

Through this notebook, we have successfully implemented state estimation and prediction using playback data. Utilizing the `BatteryCircuit` model from the `progpy` library, we were able to estimate and predict states by iterating through our playback data.

Our results include the predicted future values (inputs, states, outputs, and event_states) with their associated uncertainty, the predicted time of event occurrence, various prediction metrics, and illustrative figures.

Our final results, such as the prediction of the 'EOD' (End of Discharge) event, provide valuable insights into the performance of our battery system.

For more information, please refer to our ProgPy [Documentation](https://nasa.github.io/progpy/index.html).